In [1]:
# https://github.com/stevenhalim/cpbook-code/blob/master/ch8/maxflow.cpp
# https://github.com/stevenhalim/cpbook-code/blob/master/ch9/mcmf.cpp
import sys 
INF = 1e18

In [40]:
class graph:
    class edge:
        def __init__(self, node, cap, flow):
            self.node, self.cap, self.flow = node, cap, flow
        def __repr__(self):
            return f'{self.node}:{self.cap}@{self.flow}'
    def __init__(self, size):
        self.nodes = [[] for _ in range(size)]
        self.edges = []
    def add(self, u, v, weight, directed = True):
        if u == v: return
        self.edges.append(graph.edge(v, weight, 0))
        self.nodes[u].append(len(self.edges) - 1)
        self.edges.append(graph.edge(u, weight if not directed else 0, 0))
        self.nodes[v].append(len(self.edges) - 1)

    # Find augmenting paths
    def bfs(self, source, sink):
        self.dist = [-1] * len(self.nodes)
        self.prev = [(-1, -1)] * len(self.nodes)
        queue = [source]
        self.dist[source] = 0
        while queue:
            node = queue.pop(0)
            if node == sink: break
            for ei in self.nodes[node]:
                e = self.edges[ei]
                if e.cap - e.flow > 0 and self.dist[e.node] == -1:
                    self.dist[e.node] = self.dist[node] + 1
                    queue.append(e.node)
                    self.prev[e.node] = (node, ei)
        return self.dist[sink] != -1
    
    # Edmonds-Karp algorithm
    def send_flow(self, source, sink, f = INF):
        if source == sink: return f
        node, ei = self.prev[sink]
        e = self.edges[ei]
        pushed = self.send_flow(source, node, min(f, e.cap - e.flow))
        self.edges[ei].flow += pushed
        self.edges[ei ^ 1].flow -= pushed
        return pushed
    def edmonds_karp(self, source, sink):
        max_flow = 0
        while self.bfs(source, sink):
            if flow := self.send_flow(source, sink):
                max_flow += flow
            else:
                break
        return max_flow

g = graph(5)
g.add(0, 2, 100); g.add(0, 3, 50)
g.add(2, 3, 50);  g.add(2, 4, 50); g.add(2, 1, 50)
g.add(3, 4, 100)
g.add(4, 1, 75)
print(g.edmonds_karp(0, 1))

g = graph(5)
g.add(0, 2, 100); g.add(0, 3, 50)
g.add(2, 4, 5); g.add(2, 1, 5)
g.add(3, 4, 100)
g.add(4, 1, 75)
print(g.edmonds_karp(0, 1))

125
60


In [41]:
class graph:
    class edge:
        def __init__(self, node, cap, flow):
            self.node, self.cap, self.flow = node, cap, flow
        def __repr__(self):
            return f'{self.node}:{self.cap}@{self.flow}'
    def __init__(self, size):
        self.nodes = [[] for _ in range(size)]
        self.edges = []
    def add(self, u, v, weight, directed = True):
        if u == v: return
        self.edges.append(graph.edge(v, weight, 0))
        self.nodes[u].append(len(self.edges) - 1)
        self.edges.append(graph.edge(u, weight if not directed else 0, 0))
        self.nodes[v].append(len(self.edges) - 1)

    # Find augmenting paths
    def bfs(self, source, sink):
        self.dist = [-1] * len(self.nodes)
        self.prev = [(-1, -1)] * len(self.nodes)
        queue = [source]
        self.dist[source] = 0
        while queue:
            node = queue.pop(0)
            if node == sink: break
            for ei in self.nodes[node]:
                e = self.edges[ei]
                if e.cap - e.flow > 0 and self.dist[e.node] == -1:
                    self.dist[e.node] = self.dist[node] + 1
                    queue.append(e.node)
                    self.prev[e.node] = (node, ei)
        return self.dist[sink] != -1
    
    # Dinitz's algorithm
    def dfs(self, source, sink, f = INF):
        if source == sink or f == 0: return f
        for i in range(self.last[source], len(self.nodes[source])):
            e = self.edges[self.nodes[source][i]]
            if self.dist[e.node] != self.dist[source] + 1: # not in layer graph!
                continue
            if pushed := self.dfs(e.node, sink, min(f, e.cap - e.flow)):
                e.flow += pushed
                self.edges[self.nodes[source][i] ^ 1].flow -= pushed
                self.last[source] = i + 1
                return pushed
        self.last[source] = len(self.nodes[source])
        return 0
    def dinitz(self, source, sink):
        max_flow = 0
        while self.bfs(source, sink):
            self.last = [0] * len(self.nodes)
            while flow := self.dfs(source, sink):
                max_flow += flow
        return max_flow

g = graph(5)
g.add(0, 2, 100); g.add(0, 3, 50)
g.add(2, 3, 50);  g.add(2, 4, 50); g.add(2, 1, 50)
g.add(3, 4, 100)
g.add(4, 1, 75)
print(g.dinitz(0, 1))

g = graph(5)
g.add(0, 2, 100); g.add(0, 3, 50)
g.add(2, 4, 5); g.add(2, 1, 5)
g.add(3, 4, 100)
g.add(4, 1, 75)
print(g.dinitz(0, 1))

125
60


In [38]:
class graph:
    class edge:
        def __init__(self, node, cap, flow, cost):
            self.node, self.cap, self.flow, self.cost = node, cap, flow, cost
    def __init__(self, size):
        self.nodes = [[] for _ in range(size)]
        self.edges = []
    def add(self, u, v, weight, cost, directed = True):
        if u == v: return
        self.edges.append(graph.edge(v, weight, 0, cost))
        self.nodes[u].append(len(self.edges) - 1)
        self.edges.append(graph.edge(u, weight if not directed else 0, 0, -cost))
        self.nodes[v].append(len(self.edges) - 1)

    # Shortest paths (Bellman-Ford)
    def spfa(self, source, sink):
        self.dist = [INF] * len(self.nodes)
        queue = [source]
        self.dist[source] = 0
        self.visited[source] = True
        while queue:
            node = queue.pop(0)
            self.visited[node] = False
            for ei in self.nodes[node]:
                e = self.edges[ei]
                if e.cap - e.flow > 0 and self.dist[e.node] > self.dist[node] + e.cost:
                    self.dist[e.node] = self.dist[node] + e.cost
                    if not self.visited[e.node]:
                        queue.append(e.node)
                        self.visited[e.node] = True
        return self.dist[sink] != INF

    # Min-cost Max-flow algorithm
    def dfs(self, source, sink, f = INF):
        if source == sink or f == 0: return f
        self.visited[source] = True
        for i in range(self.last[source], len(self.nodes[source])):
            e = self.edges[self.nodes[source][i]]
            if self.visited[e.node] or self.dist[e.node] != self.dist[source] + e.cost: # not in layer graph!
                continue
            if pushed := self.dfs(e.node, sink, min(f, e.cap - e.flow)):
                self.total_cost += pushed * e.cost
                e.flow += pushed
                self.edges[self.nodes[source][i] ^ 1].flow -= pushed
                self.visited[source] = False
                self.last[source] = i + 1
                return pushed
        self.visited[source] = False
        self.last[source] = len(self.nodes[source])
        return 0
    def mcmf_dinitz(self, source, sink):
        self.total_cost = 0
        self.visited = [False] * len(self.nodes)
        max_flow = 0
        while self.spfa(source, sink):
            self.last = [0] * len(self.nodes)
            while flow := self.dfs(source, sink):
                max_flow += flow
        return max_flow, self.total_cost

    # Model costs as bipartite graph
    def assignment(prices):
        nrows = len(prices)
        ncols = len(prices[0])
        g = graph(nrows + ncols + 2)
        for r in range(nrows):
            g.add(0, r + 1, 1, 0)
        for r in range(nrows):
            for c in range(ncols):
                g.add(r + 1, nrows + c + 1, 1, prices[r][c])
        for c in range(ncols):
            g.add(nrows + c + 1, nrows + ncols + 1, 1, 0)
        return g.mcmf_dinitz(0, nrows + ncols + 1)

graph.assignment([
    # j1   j2   j3
    [108, 125, 150], # contractor 1
    [150, 135, 175], # contractor 2
    [122, 148, 250]  # contractor 3
])

(3, 407)